In [2]:
%%writefile Global_Solver_Error_Estimator.py 
import bempp.api, numpy as np
from math import pi
import os , time

from Grid_Maker import *

In [50]:
def Solv(mol_name , mesh_density, probe_radius=1.4 ,  stern_thickness=0 , min_area = 0 \
         , save_Results = True ,  create_Results =True):
    
    start_time = time.time()
    cont_itera = 0
    
    pqr_to_xyzr(mol_name , stern_thickness , method = 'amber' )
    xyzr_to_msh(mol_name , mesh_density , probe_radius , stern_thickness , min_area , Mallador = 'MSMS')

    path1 = os.path.join('Molecule',mol_name)
    triangle_areas( path1 , mol_name , mesh_density )

    bempp.api.set_ipython_notebook_viewer()
    bempp.api.PLOT_BACKEND = "ipython_notebook"


    path = path1+'/'

    grid_name_File =  os.path.join(path,mol_name + '_'+str(mesh_density)+'.msh')

    grid = bempp.api.import_grid(grid_name_File)
    #grid.plot()
    
    if save_Results: 
        if create_Results == False:
            Results_Text = open(os.path.join(path1 , 'Resultados.txt' )  ,'w+')
        else:
            Results_Text_Aux = open(os.path.join(path1 , 'Resultados.txt' ) ).read()
            Results_Text = open(os.path.join(path1 , 'Resultados.txt' )  ,'w+')
            Results_Text.write(Results_Text_Aux)

    q, x_q = np.empty(0), np.empty((0,3))

    pqr_file = os.path.join(path,mol_name+'.pqr')
    charges_file = open( pqr_file , 'r').read().split('\n')

    for line in charges_file:
        line = line.split()
        if len(line)==0: continue
        if line[0]!='ATOM': continue
        q = np.append( q, float(line[8]))
        x_q = np.vstack( ( x_q, np.array(line[5:8]).astype(float) ) )    

    from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

    dirichl_space = bempp.api.function_space(grid,  "DP", 0)
    neumann_space = bempp.api.function_space(grid,  "DP", 0) 
    dual_to_dir_s = bempp.api.function_space(grid,  "DP", 0)

    identity = sparse.identity(     dirichl_space, dirichl_space, dual_to_dir_s)
    slp   = laplace.single_layer(neumann_space, dirichl_space, dual_to_dir_s)
    dlp   = laplace.double_layer(dirichl_space, dirichl_space, dual_to_dir_s)

    def zero_i(x, n, domain_index, result):
        result[:] = 0

    def u_s_G(x,n,domain_index,result):
        global q,x_q,ep_m,C
        result[:] = C / (4.*np.pi*ep_m)  * np.sum( q / np.linalg.norm( x - x_q, axis=1 ) )

    def du_s_G(x,n,domain_index,result):
        global q,x_q,ep_m,C
        result[:] = -C/(4.*np.pi*ep_m)  * np.sum( np.dot( x-
                                x_q , n)  * q / np.linalg.norm( x - x_q, axis=1 )**3 )

    u_s  = bempp.api.GridFunction(dirichl_space, fun=u_s_G)
    du_s = bempp.api.GridFunction(neumann_space, fun=du_s_G)

    from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

    identity = sparse.identity(     dirichl_space, dirichl_space, dual_to_dir_s)
    slp_in   = laplace.single_layer(neumann_space, dirichl_space, dual_to_dir_s)
    dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dual_to_dir_s)

    sol, info,it_count = bempp.api.linalg.gmres( slp_in, -(dlp_in+0.5*identity)*u_s , return_iteration_count=True, tol=1e-4)
    print("The linear system for u_h was solved in {0} iterations".format(it_count))
    
    cont_itera = cont_itera + it_count

    u_h = -u_s
    du_h = sol

    k_p = k # kappa'

    identity = sparse.identity(     dirichl_space, dirichl_space, dual_to_dir_s)
    slp_in   = laplace.single_layer(neumann_space, dirichl_space, dual_to_dir_s)
    dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dual_to_dir_s)

    slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dual_to_dir_s, k_p)
    dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dual_to_dir_s, k_p)

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0, 0] = 0.5*identity + dlp_in
    blocked[0, 1] = -slp_in
    blocked[1, 0] = 0.5*identity - dlp_out
    blocked[1, 1] = (ep_m/ep_s)*slp_out


    zero = bempp.api.GridFunction(dirichl_space, fun=zero_i)

    rhs = [ zero ,  -slp_out *(ep_m/ep_s)* (du_s+du_h)]

    sol, info,it_count = bempp.api.linalg.gmres( blocked , rhs, return_iteration_count=True, tol=1e-4)
    print("The linear system for u_r was solved in {0} iterations".format(it_count))
    u_r , du_r = sol
    
    cont_itera = cont_itera + it_count

    from bempp.api.operators.potential import laplace as lp

    slp_in_O = lp.single_layer(neumann_space, x_q.transpose()) 
    dlp_in_O = lp.double_layer(dirichl_space, x_q.transpose())

    u_r_O = slp_in_O * du_r  -  dlp_in_O * u_r
    u_h_O = slp_in_O * du_h  +  dlp_in_O * u_s

    terms =  u_r_O + u_h_O

    S     = 0.5 * 4. * np.pi * 332.064 * np.sum(q * terms).real
    print("Three Term Splitting : {:7.8f} [kCal/mol] ".format(S) )
    
    if save_Results: Results_Text.write( str(S)+' ')

    # A D J O I N T
    # We build the right side

    def PHI_times_G_L(x, n, domain_index, result):
        global q,x_q,ep_m,C , k
        result[:] = 1. / (4.*np.pi)  * np.sum( q  / np.linalg.norm( x - x_q, axis=1 ) )

    identity = sparse.identity(     dirichl_space, dirichl_space, dual_to_dir_s)
    slp_in   = laplace.single_layer(neumann_space, dirichl_space, dual_to_dir_s)
    dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dual_to_dir_s)

    slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dual_to_dir_s, k_p)
    dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dual_to_dir_s, k_p)

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0, 0] = 0.5*identity + dlp_in
    blocked[0, 1] = -slp_in
    blocked[1, 0] = 0.5*identity - dlp_out
    blocked[1, 1] = (ep_m/ep_s)*slp_out

    zero = bempp.api.GridFunction(dirichl_space , fun=zero_i)
    P_GL = bempp.api.GridFunction(dirichl_space, fun=PHI_times_G_L)
    rs_r = [P_GL , zero]

    sol_r, info,it_count = bempp.api.linalg.gmres( blocked, rs_r , return_iteration_count=True, tol=1e-4)
    print("The linear system for phi_r was solved in {0} iterations".format(it_count))
    phi_r , dphi_r = sol_r
    
    cont_itera = cont_itera + it_count

    #phi_r.plot()

    rs_h = P_GL - (  (0.5*identity + dlp_in)*phi_r - slp_in * dphi_r  )

    sol_h, info,it_count = bempp.api.linalg.gmres( -dlp_in, rs_h, return_iteration_count=True, tol=1)
    print("The linear system for phi_h was solved in {0} iterations".format(it_count))

    cont_itera = cont_itera + it_count
    
    dphi_h = zero 
    phi_h  = zero
    
    

    # Se definen las integrales de superficie, válido para espacios DP0 unicamente.
    def Integral_superficie( A , B ):
        A = A.real.coefficients
        B = B.real.coefficients
        Area_F = open(path+'triangleAreas_'+str(mesh_density)+'.txt').readlines()
        Areas  = np.zeros(len(Area_F))
        for i in range(len(Area_F)):
            Areas[i] = float(Area_F[i][:-1]) 
        return A*B*Areas

    # Se abre el archivo de texto que guarda las áreas por elemento
    area_txt = open('Molecule/{0:s}/triangleAreas_{1:s}.txt'.format(mol_name , str(mesh_density) ) ).read().split('\n')
    areas    = [float(i) for i in area_txt[:-1]]

    slp_in_O = lp.single_layer(neumann_space, x_q.transpose()) 
    dlp_in_O = lp.double_layer(dirichl_space, x_q.transpose())

    phi_r_O = slp_in_O * dphi_r  -  dlp_in_O * phi_r
    phi_h_O = slp_in_O * dphi_h  +  dlp_in_O * phi_h

    terms =  u_r_O + u_h_O

    S_phi     = np.sum(q * terms).real

    E_m = np.sum( Integral_superficie( -ep_m * u_h , dphi_h ) )

    E_r_1 =  np.sum( Integral_superficie( -ep_m * du_s , phi_r ) ) + \
                np.sum( Integral_superficie( u_h  , ep_m * dphi_r ) ) 
    
    elapsed_time = time.time() - start_time
    
    print('Error: {:.8f}'.format(E_r_1) )
    
    N_el = dirichl_space.global_dof_count
    
    txt =  ' '.join(map( str, [E_r_1, elapsed_time, it_count, mesh_density, N_el] ))
    
    if save_Results: Results_Text.write( txt +'\n')
    
    Results_Text.close()
    
    return None

In [52]:
ep_m = 4.
ep_s = 80.
k = 0.125

e_c = 1.60217662e-19 # [C] - proton charge
k_B = 1.38064852e-23 # [m2 kg s-2 K-1]
T   = 298. # [K]  
C = 1. # e_c/(k_B*T)

mol_name = 'arg' 

d_max = 21.0
d_min = 2.0
r = 2.0
listing = np.arange(d_min , d_max , r)
n , c= len(listing) , 0.

cont_Write =False
for mesh_density in listing:
    c+=1.
    print( '{:.1f}%'.format(c/n*100.))
    if c == n:
        print('¡Last iteration!')
    Solv(mol_name , mesh_density , create_Results=cont_Write)
    cont_Write =True
    print('--------------------------------------------------------------')
print('Done')

10.0%
.xyzr File from .pqr ready.
Normal .vert & .face Done
Normal .msh Done
Mesh Ready
The linear system for u_h was solved in 215 iterations
The linear system for u_r was solved in 1065 iterations
Three Term Splitting : -74.78181668 [kCal/mol] 
The linear system for phi_r was solved in 121 iterations
The linear system for phi_h was solved in 0 iterations
Error: 0.13720610
--------------------------------------------------------------
20.0%
.xyzr File from .pqr ready.
Normal .vert & .face Done
Normal .msh Done
Mesh Ready
The linear system for u_h was solved in 95 iterations
The linear system for u_r was solved in 195 iterations
Three Term Splitting : -65.42225667 [kCal/mol] 
The linear system for phi_r was solved in 98 iterations
The linear system for phi_h was solved in 0 iterations
Error: 0.13470584
--------------------------------------------------------------
30.0%
.xyzr File from .pqr ready.
Normal .vert & .face Done
Normal .msh Done
Mesh Ready
The linear system for u_h was solve

In [41]:
n , c= len(listing), 1.

for mesh_density in listing:
    print( '{:.1f}%'.format(c/n*100.))
    if c == n:
        print('¡Last iteration!')

(1.0, 3)
33.3%
(1.0, 3)
33.3%
(1.0, 3)
33.3%


In [49]:
print("2^{3}")

2^{3}
